In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import os
import sys

import pandas as pd

In [2]:
# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [34]:
# 환경 세팅
# conf_spark = SparkConf().set("spark.driver.host", "127.0.0.1") # local 환경임을 명시적으로 지정해준다. 
# sc = SparkContext(conf=conf_spark)

In [5]:
spark = SparkSession.builder.master("local[1]").appName("kidney_disease").getOrCreate()

### local data read

In [6]:
data = spark.read.option("header", True).csv("./kidney_disease.csv") # header가 있는 경우, option 구문 추가 

In [32]:
data.show()

+---+----+-----+-----+----+----+--------+--------+----------+----------+-----+-----+----+-----+----+----+----+-----+----+---+---+---+-----+---+---+--------------+
| id| age|   bp|   sg|  al|  su|     rbc|      pc|       pcc|        ba|  bgr|   bu|  sc|  sod| pot|hemo| pcv|   wc|  rc|htn| dm|cad|appet| pe|ane|classification|
+---+----+-----+-----+----+----+--------+--------+----------+----------+-----+-----+----+-----+----+----+----+-----+----+---+---+---+-----+---+---+--------------+
|  0|48.0| 80.0| 1.02| 1.0| 0.0|    null|  normal|notpresent|notpresent|121.0| 36.0| 1.2| null|null|15.4|  44| 7800| 5.2|yes|yes| no| good| no| no|           ckd|
|  1| 7.0| 50.0| 1.02| 4.0| 0.0|    null|  normal|notpresent|notpresent| null| 18.0| 0.8| null|null|11.3|  38| 6000|null| no| no| no| good| no| no|           ckd|
|  2|62.0| 80.0| 1.01| 2.0| 3.0|  normal|  normal|notpresent|notpresent|423.0| 53.0| 1.8| null|null| 9.6|  31| 7500|null| no|yes| no| poor| no|yes|           ckd|
|  3|48.0| 70.0|1.005|

In [7]:
# 각 컬럼별로 결측값 개수를 확인함 
# 만약 결측값 코딩이 0으로 되어 있는 경우, filter 조건을 변경하면 된다. 
result = pd.DataFrame({'variable':[], 'missing_count':[]})
for col in data.columns:
    df_new = pd.DataFrame({'variable':[], 'missing_count':[]})
    df_new.at[0, 'variable'] = col
    df_new.at[0, 'missing_count'] = data.filter(data[col].isNull()).count()
    result = pd.concat([result, df_new], axis=0)
    
result.astype({'missing_count': 'int32'}) # missing_count 을 int 형으로 변환함 
result['missing_rate'] = result['missing_count'] / data.count() # 결측률 계산 
result

,variable,missing_count,missing_rate
0,id,0.0,0.0000
0,age,9.0,0.0225
0,bp,12.0,0.0300
0,sg,47.0,0.1175
0,al,46.0,0.1150
0,su,49.0,0.1225
0,rbc,152.0,0.3800
0,pc,65.0,0.1625
0,pcc,4.0,0.0100
0,ba,4.0,0.0100
